In [57]:
import pickle
from sklearn.neighbors import NearestNeighbors
import numpy as np
import cv2
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image

# Load the data
outfit_df = pickle.load(open('outfit_df.pkl', 'rb'))
jewelry_df = pickle.load(open('jewelry_df.pkl', 'rb'))



In [58]:
# Function to recommend jewelry based on features
def recommend_jewelry(features, feature_list):
    neighbors = NearestNeighbors(n_neighbors=10, algorithm='brute', metric='euclidean')
    neighbors.fit(feature_list)
    distances, indices = neighbors.kneighbors([features])
    filtered_indices = indices[0]
    return filtered_indices



In [59]:
# Function to recommend similar items
def recommend_similar_items(features):
    neighbors = NearestNeighbors(n_neighbors=10, algorithm='brute', metric='euclidean')
    neighbors.fit(outfit_df['features'].tolist())
    distances, indices = neighbors.kneighbors([features])
    return indices[0]



In [60]:
# Function to extract features from an image using ResNet50
def extract_features(img_path):
    model = ResNet50(weights='imagenet', include_top=False, pooling='avg')
    img = image.load_img(img_path, target_size=(224, 224))
    img_data = image.img_to_array(img)
    img_data = np.expand_dims(img_data, axis=0)
    img_data = preprocess_input(img_data)
    features = model.predict(img_data)
    return features[0]



In [61]:
# Function to calculate Recall@K
def recall_at_k(recommended_indices, relevant_indices, k):
    recommended_k = recommended_indices[:k]
    relevant_set = set(relevant_indices)
    recommended_set = set(recommended_k)
    intersection = recommended_set.intersection(relevant_set)
    recall = len(intersection) / len(relevant_set) if relevant_set else 0.0
    return recall



In [62]:
# Function to calculate Precision@K
def precision_at_k(recommended_indices, relevant_indices, k):
    recommended_k = recommended_indices[:k]
    relevant_set = set(recommended_indices)
    recommended_set = set(recommended_k)
    intersection = recommended_set.intersection(relevant_set)
    precision = len(intersection) / k if k else 0.0
    return precision



In [63]:
# Function to calculate F1@K
def f1_at_k(recommended_indices, relevant_indices, k):
    recall = recall_at_k(recommended_indices, relevant_indices, k)
    precision = precision_at_k(recommended_indices, relevant_indices, k)
    if recall + precision == 0:
        return 0.0
    return 2 * (recall * precision) / (recall + precision)



In [64]:
# Function to calculate accuracy
def accuracy_at_k(recommended_indices, relevant_indices, k):
    recommended_k = recommended_indices[:k]
    relevant_set = set(relevant_indices)
    recommended_set = set(recommended_k)
    intersection = recommended_set.intersection(relevant_set)
    accuracy = len(intersection) / k if k else 0.0
    return accuracy



In [65]:
user_image_path = input("Enter the path of outfit image: ").strip()

# Ensure the path is handled correctly
user_image_path = user_image_path.replace('"', '')



Enter the path of outfit image: "C:\Users\pragy\OneDrive\Desktop\ml files\images\22563.jpg"


In [66]:
try:
    query_features = extract_features(user_image_path)
    
    # Print extracted features for debugging
    print("Extracted Features: ", query_features)
    
    # Ask the user for additional information
    gender = input("Enter the gender of the outfit (e.g., Men/Women): ").capitalize()
    article_type = input("Enter the article type of the outfit (e.g., Dresses, Lehenga Choli, Salwar and Dupatta, Sarees, Kurta Sets): ").title()
    
    is_womens_outfit = (gender == 'Women')
    is_apparel = (article_type in ['Dresses', 'Lehenga Choli', 'Salwar and Dupatta', 'Sarees', 'Kurta Sets'])
    
    # Filter the dataset based on the provided gender and articleType
    filtered_outfit_df = outfit_df[(outfit_df['gender'] == gender) & (outfit_df['articleType'] == article_type)]
    
    if not filtered_outfit_df.empty:
        filtered_indices = filtered_outfit_df.index.tolist()
        filtered_features = filtered_outfit_df['features'].tolist()
        
        # Recommend jewelry based on outfit features
        recommended_indices = recommend_jewelry(query_features, jewelry_df['features'].tolist())
        
        if len(recommended_indices) == 0:
            print("No recommended jewelry items found.")
        else:
            # Display the recommended jewelry images
            for i in recommended_indices[:5]:  # Displaying top 5 recommendations
                temp_img = cv2.imread(jewelry_df.loc[i, 'productImage'])
                cv2.imshow('Recommended Jewelry', cv2.resize(temp_img, (512, 512)))
                cv2.waitKey(0)
                cv2.destroyAllWindows()
                
    else:
        print("No matching outfits found for the given gender and article type.")
        print("Recommending similar items based on outfit features...")
        
        # Recommend similar items based on outfit features
        similar_indices = recommend_similar_items(query_features)
        
        if len(similar_indices) == 0:
            print("No similar items found.")
        else:
            # Display the recommended similar outfit images
            for i in similar_indices[:5]:  # Displaying top 5 recommendations
                temp_img = cv2.imread(outfit_df.loc[i, 'imagePath'])
                cv2.imshow('Recommended Similar Outfit', cv2.resize(temp_img, (512, 512)))
                cv2.waitKey(0)
                cv2.destroyAllWindows()

        # Map the predicted indices to their IDs
        predicted_ids = [outfit_df.loc[i, 'id'] for i in similar_indices[:5]]
        print(f"Predicted Similar Indices: {similar_indices[:5]}")
        print(f"Predicted Similar IDs: {predicted_ids}")
    
    # Get accurate indices from user
    accurate_indices_input = input("Enter the accurate indices separated by commas (e.g., 1,2,3,4,5): ").strip()
    accurate_indices = list(map(int, accurate_indices_input.split(',')))
    
    # Calculate evaluation metrics
    k = 5  # Number of top recommendations to consider

    # Use predicted_indices for jewelry or similar items
    if is_womens_outfit and is_apparel:
        predicted_ids = [jewelry_df.loc[i, 'id'] for i in recommended_indices[:5]]
    else:
        predicted_ids = [outfit_df.loc[i, 'id'] for i in similar_indices[:5]]
    
    recall = recall_at_k(predicted_ids, accurate_indices, k)
    precision = precision_at_k(predicted_ids, accurate_indices, k)
    f1_score = f1_at_k(predicted_ids, accurate_indices, k)
    accuracy = accuracy_at_k(predicted_ids, accurate_indices, k)

    print(f"Recall@{k}: {recall}")
    print(f"Precision@{k}: {precision}")
    print(f"F1@{k}: {f1_score}")
    print(f"Accuracy@{k}: {accuracy}")

except Exception as e:
    print(f"Error: {e}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
Extracted Features:  [0.         0.3324459  0.02588665 ... 0.05161736 0.46936205 0.13284789]
Enter the gender of the outfit (e.g., Men/Women): unisex
Enter the article type of the outfit (e.g., Dresses, Lehenga Choli, Salwar and Dupatta, Sarees, Kurta Sets): bag
No matching outfits found for the given gender and article type.
Recommending similar items based on outfit features...
Predicted Similar Indices: [ 2544  4210 25341  9527 33920]
Predicted Similar IDs: [22563, 22562, 22561, 25430, 18796]
Enter the accurate indices separated by commas (e.g., 1,2,3,4,5): 22563,22562,22561,25430,22565
Recall@5: 0.8
Precision@5: 1.0
F1@5: 0.888888888888889
Accuracy@5: 0.8
